In [ ]:
import yt_dlp


# 유튜브 URL
url = "https://youtu.be/zVLjWIftX_o?si=zFNesrc3CTkB9pCN"

# 다운로드 옵션 (오디오만 다운로드)
ydl_opts = {
    "format": "bestaudio/best",
    "postprocessors": [
        {
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192",
        }
    ],
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

In [ ]:
from moviepy.editor import AudioFileClip
import os

In [ ]:
from transformers import pipeline

whisper_small = pipeline("automatic-speech-recognition", "openai/whisper-small")

In [ ]:
# 오디오 파일을 30초 단위로 나누는 함수
def split_audio(file_path, chunk_length=30):
    audio = AudioFileClip(file_path)
    total_duration = int(audio.duration)
    audio_chunks = []

    for start in range(0, total_duration, chunk_length):
        end = min(start + chunk_length, total_duration)
        chunk = audio.subclip(start, end)
        chunk_path = f"chunk_{start}_{end}.mp3"
        chunk.write_audiofile(chunk_path)
        audio_chunks.append(chunk_path)

    return audio_chunks


# 각 청크에서 텍스트 추출
def transcribe_audio_chunks(chunk_files):
    transcriptions = []
    for chunk_file in chunk_files:
        result = whisper_small(chunk_file, generate_kwargs={"language": "english"})
        transcriptions.append(result["text"])
        os.remove(chunk_file)  # 임시 파일 삭제
    return transcriptions


# 전체 텍스트를 하나로 합침
def merge_transcriptions(transcriptions):
    return " ".join(transcriptions)


# 오디오 파일 경로
audio_file = "./Relating addition and subtraction ｜ Basic addition and subtraction ｜ Early Math ｜ Khan Academy [zVLjWIftX_o].mp3"

# 오디오 파일을 30초 단위로 나누고 텍스트 추출
chunks = split_audio(audio_file)
transcriptions = transcribe_audio_chunks(chunks)
final_transcription = merge_transcriptions(transcriptions)

# 최종 결과 출력
print(final_transcription)

In [ ]:
import openai

# OpenAI API 키 설정
openai.api_key = ""

# 번역 및 요약할 긴 영어 텍스트
text_to_translate_and_summarize = """I have two green balls and three purple balls. So how many balls do I have altogether? Well, that's going to be two purple, sorry, two green balls plus three purple balls is going to be five balls altogether. And you see that right over there? There's five balls altogether. We can count them. One, two, three, four, five. Now what if we did something a little different? Let's start with the five balls. So.  We're gonna start with the five balls, and you can count them again, one, two, three, four, five. And now let's take away two, let's subtract, let's subtract two of these balls. So let's take away one and two. Well then, what are we left with? Well then, we are left with one, two, three balls. We're left with three balls. And you might already see a pattern here. Two plus three is...  equal to five, but if you start with five and you take away two, you're going to be left with three. Look at how this is arranged and look at how this is arranged. Two plus three is equal to five, but five minus two is three. Five minus two is three. And it works the other way around. If I start with five, if I start with five, and if I were to take away three, so if I subtract one, two, three, what am I left with? Well I'm left.  with one, two. I am left with two. So two plus three is five. Three plus two is five. We see that there, which is another way of saying that five minus two is three, or five minus three is two. Take a look at this for a little bit. This is a really, really important pattern, or a really important idea to understand in math. If you add two numbers and you get, say, if you add, say, a two and a three, or you add two numbers and you get another number, Well, if you take...  If you take that number and you subtract one of these, you're going to get the other number."""

# GPT-4를 사용하여 텍스트를 주제별로 나누고 번역 및 요약
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": f"Translate this text to Korean, divide it by topic, and summarize each topic: {text_to_translate_and_summarize}",
        },
    ],
)

# 번역 및 요약된 결과 출력
print(response["choices"][0]["message"]["content"])